In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from multiprocessing import Process, Queue
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
from scipy.misc import imsave
import scipy.stats
import random
from sklearn.metrics import accuracy_score

from problog import get_evaluatable
from problog.program import PrologString, SimpleProgram
from problog.core import ProbLog
from problog.logic import Constant,Var,Term,AnnotatedDisjunction
from problog.learning import lfi

%matplotlib notebook

In [2]:
model = """
t(0.5)::gender.
t(0.25)::age_1;t(0.25)::age_2;t(0.25)::age_3;t(0.25)::age_4.
t(0.5)::smoke.
t(0.5)::alcohol.

t(0.33)::bmi_1;t(0.33)::bmi_2;t(0.33)::bmi_3.
t(0.5)::ldl.
t(0.33)::blood_pressure_1;t(0.33)::blood_pressure_2;t(0.33)::blood_pressure_3.

t(_)::blood_pressure_1:-gender.
t(_)::blood_pressure_1:-age_1.
t(_)::blood_pressure_1:-age_2.
t(_)::blood_pressure_1:-age_3.
t(_)::blood_pressure_1:-age_4.
t(_)::blood_pressure_1:-smoke.
t(_)::blood_pressure_1:-alcohol.

t(_)::blood_pressure_2:-gender.
t(_)::blood_pressure_2:-age_1.
t(_)::blood_pressure_2:-age_2.
t(_)::blood_pressure_2:-age_3.
t(_)::blood_pressure_2:-age_4.
t(_)::blood_pressure_2:-smoke.
t(_)::blood_pressure_2:-alcohol.

t(_)::blood_pressure_3:-gender.
t(_)::blood_pressure_3:-age_1.
t(_)::blood_pressure_3:-age_2.
t(_)::blood_pressure_3:-age_3.
t(_)::blood_pressure_3:-age_4.
t(_)::blood_pressure_3:-smoke.
t(_)::blood_pressure_3:-alcohol.

t(_)::ldl_t:-gender.
t(_)::ldl_t:-age_1.
t(_)::ldl_t:-age_2.
t(_)::ldl_t:-age_3.
t(_)::ldl_t:-age_4.
t(_)::ldl_t:-smoke.
t(_)::ldl_t:-alcohol.

t(_)::ldl_f:-gender.
t(_)::ldl_f:-age_1.
t(_)::ldl_f:-age_2.
t(_)::ldl_f:-age_3.
t(_)::ldl_f:-age_4.
t(_)::ldl_f:-smoke.
t(_)::ldl_f:-alcohol.

t(_)::bmi_1:-gender.
t(_)::bmi_1:-age_1.
t(_)::bmi_1:-age_2.
t(_)::bmi_1:-age_3.
t(_)::bmi_1:-age_4.
t(_)::bmi_1:-smoke.
t(_)::bmi_1:-alcohol.

t(_)::bmi_2:-gender.
t(_)::bmi_2:-age_1.
t(_)::bmi_2:-age_2.
t(_)::bmi_2:-age_3.
t(_)::bmi_2:-age_4.
t(_)::bmi_2:-smoke.
t(_)::bmi_2:-alcohol.

t(_)::bmi_3:-gender.
t(_)::bmi_3:-age_1.
t(_)::bmi_3:-age_2.
t(_)::bmi_3:-age_3.
t(_)::bmi_3:-age_4.
t(_)::bmi_3:-smoke.
t(_)::bmi_3:-alcohol.

t(_)::heart_d:-blood_pressure_1.
t(_)::heart_d:-blood_pressure_2.
t(_)::heart_d:-blood_pressure_3.
t(_)::heart_d:-ldl.
t(_)::heart_d:-bmi_1.
t(_)::heart_d:-bmi_2.
t(_)::heart_d:-bmi_3.

t(_)::glucose:-blood_pressure_1.
t(_)::glucose:-blood_pressure_2.
t(_)::glucose:-blood_pressure_3.
t(_)::glucose:-ldl.
t(_)::glucose:-bmi_1.
t(_)::glucose:-bmi_2.
t(_)::glucose:-bmi_3.
"""

In [3]:
gender = Term('gender')
age_1 = Term('age_1')
age_2 = Term('age_2')
age_3 = Term('age_3')
age_4 = Term('age_4')
smoke = Term('smoke')
alcohol = Term('alcohol')

bmi_1 = Term('bmi_1')
bmi_2 = Term('bmi_2')
bmi_3 = Term('bmi_3')
ldl = Term('ldl')
blood_pressure_1 = Term('blood_pressure_1')
blood_pressure_2 = Term('blood_pressure_2')
blood_pressure_3 = Term('blood_pressure_3')

heart_d = Term('heart_d')
glucose = Term('glucose')

In [4]:
#examples = [
#    [(gender_female, True), (age_2, True), (blood_pressure_1, True)],
#    [(gender_female, False), (age_2, False), (blood_pressure_2, False)]
#]

df= pd.read_pickle("data.pkl")

def convert_row(row):
    results = []
    
    if row["RIAGENDR"]==2.0:
        results += [(gender, False)]
    if row["RIAGENDR"]==1.0:
        results += [(gender, True)]
    if row["RIDAGEYR"]==0.0:
        results += [(age_1, True)]
    if row["RIDAGEYR"]==1.0:
        results += [(age_2, True)]
    if row["RIDAGEYR"]==2.0:
        results += [(age_3, True)]
    if row["RIDAGEYR"]==3.0:
        results += [(age_4, True)]
    if row["ALQ120Q"]==1.0:
        results += [(alcohol, True)]
    if row["ALQ120Q"]==0.0:
        results += [(alcohol, False)]
    if row["SMQ020"]==1.0:
        results += [(smoke, True)]
    if row["SMQ020"]==2.0:
        results += [(smoke, False)]
    if row["BPXDI1"]==0.0:
        results += [(blood_pressure_1, True)]
    if row["BPXDI1"]==1.0:
        results += [(blood_pressure_2, True)]
    if row["BPXDI1"]==2.0:
        results += [(blood_pressure_3, True)]
    if row["LBDLDL"]==0.0:
        results += [(ldl, True)]
    if row["LBDLDL"]==1.0:
        results += [(ldl, False)]
    if row["BMXBMI"]==0.0:
        results += [(bmi_1, True)]
    if row["BMXBMI"]==1.0:
        results += [(bmi_2, True)]
    if row["BMXBMI"]==2.0:
        results += [(bmi_3, True)]
    if row["HEARTDISEASE"]==True:
        results += [(heart_d, True)]
    if row["HEARTDISEASE"]==False:
        results += [(heart_d, False)]
    if row["LBXGLU"]==1.0:
        results += [(glucose, True)]
    if row["LBXGLU"]==0.0:
        results += [(glucose, False)]
    
    
    return results

evidence = [convert_row(df.iloc[0]), convert_row(df.iloc[3])]
#evidence = [convert_row(df.iloc[i]) for i in range(3,5)]

evidence = [[(gender, True), (age_1, True), (age_2, False), (age_3, False), (bmi_1, True)],
 [(gender, False), (age_1, False), (age_2, True), (age_3, False)]]

evidence

TypeError: 'tuple' object is not callable

In [ ]:
score, weights, atoms, iteration, lfi_problem = lfi.run_lfi(PrologString(model), examples)

In [ ]:
print (lfi_problem.get_model())